In [29]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [30]:
from huggingface_hub import notebook_login
notebook_login()

In [106]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



538


In [107]:
from datasets import Dataset, load_metric

In [108]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [109]:
len(train)

1255

In [110]:
train['label'].value_counts()

0    422
2    420
1    413
Name: label, dtype: int64

In [111]:
test['label'].value_counts()

0    181
2    180
1    177
Name: label, dtype: int64

In [112]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [113]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
1,0950c46beda335819928585f1262dfe1dca78a0b,ReactiveX-RxJava,Trying to extend the Scheduler interface accor...,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
498,869a002cae63a4e8ab52ec7f2d15d5a2cfbe0c02,drools,[DROOLS-839] fix LogicTransformer with Accumul...,0,0,0,1,9,1,1,...,0,0,0,1,0,1,0,0,0,0
1010,3ca94cabb68789cb20e37e0c9ecea7acc8b9487e,Valadoc,"libvaladoc/html: Add ""All known members inheri...",1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1583,171accb43d709bd3960187686afbdf68f5239644,aeshell$aesh,[AESH-316] - print warning instead of exceptio...,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1369,5b91442a260b42e3d47ef91a53015f2d0958a683,Delta Spike,remove unused private field\n,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,f2757568dac2a6473f054101f80ac2b8f291ce4e,Vala,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1574,16cb7d8d59add7b3ef513bba0de6a4fc07e3bc52,aeshell$aesh,"search is more complete, support forward/rever...",1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
597,14c657d448b6dd743806c4df2a321d58f4e0618e,kotlin,"Extract Function: Consider reference ""broken"" ...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1712,02668af08bebc5266b321ab1df2f45dfe656c813,duracloud$duracloud,First part of https://jira.duraspace.org/brows...,2,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


In [114]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [115]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [116]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [117]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [118]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [119]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [125]:
def compute_metrics(eval_pred):
    accuracy = load_metric('accuracy')
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [126]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

<ipython-input-118-ebcd56f8ad98>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.689454,{'precision': 0.7755999222940274},{'recall': 0.7620817843866171},{'f1': 0.7615277815046925},{'accuracy': 0.7620817843866171}


Trainer is attempting to log a value of "{'precision': 0.7755999222940274}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.7620817843866171}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7615277815046925}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7620817843866171}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=79, training_loss=0.3144996498204485, metrics={'train_runtime': 136.4919, 'train_samples_per_second': 9.195, 'train_steps_per_second': 0.579, 'total_flos': 330207339248640.0, 'train_loss': 0.3144996498204485, 'epoch': 1.0})